# Step One: Setup

### Integrating Environmental Factors into Storm Water Structure Point Layer

**Main Layer**: **Storm Water Structure Point Layer**

#### Environmental Factors:
1. **Distance to Stream**
2. **Distance to Roads**
3. **Digital Elevation Model (DEM)**


### 1. Distance to Streams

###### Source: https://www.nconemap.gov/datasets/e6b5694f81ed49c6a0f3600540f49878_0/explore?location=35.315422%2C-80.715475%2C14.61

In [25]:
streamPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\01_streamDistance\streamClipped.shp"

### 2. Distance to Roads

###### Source: https://connect.ncdot.gov/resources/gis/Pages/GIS-Data-Layers.aspx

In [26]:
roadPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\02_roadsDistance\roadsClipped.shp"

### 3. Mecklenburg DEM

###### Source: https://sdd.nc.gov/DownloadFiles.aspx?path=DEMMosaicsbyCounty/DEM50_CountywideRasters

In [27]:
demPointsPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\03_meckDEM\rasterToPoint.shp"
## The field for elevation seems to be grid_code 
## This is after taking the DEM raster into arcPro and using the raster to points function

### 4. Storm Water Structures

In [28]:
stormPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\00_theData\inlet.csv"

# Step Two: Coding

### Import Statements

In [29]:
import pandas as pd
import geopandas as gpd
import time
from shapely.geometry import Point
from rasterstats import point_query
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import osgeo
from osgeo import gdal
from shapely.ops import nearest_points
import fiona

### Structures CSV into DataFrame

In [30]:
stormpoints_df = pd.read_csv(stormPath)
stormpoints_gdf = gpd.GeoDataFrame(
    stormpoints_df,
    geometry=gpd.points_from_xy(stormpoints_df['Longitude'], stormpoints_df['Latitude']),
    crs="EPSG:2264"
)

### Loading Road GDF

In [31]:
roads_gdf = gpd.read_file(roadPath)

target_crs = "EPSG:2264"
stormpoints_gdf = stormpoints_gdf.to_crs(target_crs)
print(stormpoints_gdf.crs)
roads_gdf = roads_gdf.to_crs(target_crs)
# roads_gdf was originally loaded as 6360 but GeoPandas can't handle Compound CRS
print(roads_gdf.crs)



c:\Users\rcarte64\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\pyogrio\raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(


EPSG:2264
EPSG:2264


### Distance to Nearest Road

In [32]:
def min_distance_to_roads(stormpoint, roads):
    return roads.distance(stormpoint).min()

stormpoints_gdf['nearest_road'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_roads(point, roads_gdf)
)

### Loading Streams GDF

In [33]:
streams_gdf = gpd.read_file(streamPath)
streams_gdf = streams_gdf.to_crs(target_crs)

### Distance to Nearest Stream

In [34]:
def min_distance_to_streams(stormpoint,streams):
    return streams.distance(stormpoint).min()

stormpoints_gdf['nearest_stream'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_streams(point, streams_gdf)
)
    

### DEM


In [35]:
dem_gdf = gpd.read_file(demPointsPath)
print (dem_gdf.crs)
dem_gdf = dem_gdf.to_crs(target_crs)
print (dem_gdf.crs)

PROJCS["NAD_1983_2011_StatePlane_North_Carolina_FIPS_3200_Ft_US",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",33.75],PARAMETER["central_meridian",-79],PARAMETER["standard_parallel_1",34.3333333333333],PARAMETER["standard_parallel_2",36.1666666666667],PARAMETER["false_easting",2000000],PARAMETER["false_northing",0],UNIT["Foot_US",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
EPSG:2264


In [36]:
# dem_sindex = dem_gdf.sindex
# def find_nearest_elevation(point, dem_df, dem_sindex):
#     possible_matches_index = list(dem_sindex.nearest(point.bounds, 1))
#     nearest_geom = dem_gdf.iloc[possible_matches_index].geometry.iloc[0]

#     #Get value from grid_code field
#     nearest_dem_point = dem_gdf[dem_gdf.geometry == nearest_geom].iloc[0]
#     return nearest_dem_point['grid_code']

In [37]:
# #Removing non-geometry objects
# stormpoints_gdf = stormpoints_gdf[stormpoints_gdf.geometry.notnull()]
# dem_gdf = dem_gdf[dem_gdf.geometry.notnull()]

# # Fixing Invalid Geoms
# stormpoints_gdf["geometry"] = stormpoints_gdf.geometry.buffer(0)
# dem_gdf["geometry"] = dem_gdf.geometry.buffer(0)

# #Nearest elevation -> each storm point
# stormpoints_gdf['elevation'] = stormpoints_gdf.geometry.apply(
#     lambda point: find_nearest_elevation(point, dem_gdf, dem_sindex)
# )

In [38]:
# print(stormpoints_gdf.geometry.is_valid)
# print(dem_gdf.geometry.is_valid)

In [53]:
# Verify input types
print("Type of stormpoints_gdf:", type(stormpoints_gdf))
print("Type of dem_gdf:", type(dem_gdf))

# Verify geometry column
print("Geometry column in stormpoints_gdf:", stormpoints_gdf.geometry.name)
print("Geometry column in dem_gdf:", dem_gdf.geometry.name)

# Verify the CRS
print("CRS of stormpoints_gdf:", stormpoints_gdf.crs)
print("CRS of dem_gdf:", dem_gdf.crs)

# Ensure stormpoints_gdf is a GeoDataFrame
if not isinstance(stormpoints_gdf, gpd.GeoDataFrame):
    stormpoints_gdf = gpd.GeoDataFrame(stormpoints_gdf, geometry='geometry', crs=target_crs)

print("Previewing the dem_gdf 'grid_code' column...")
print(dem_gdf[['geometry', 'grid_code']].head())



Type of stormpoints_gdf: <class 'geopandas.geodataframe.GeoDataFrame'>
Type of dem_gdf: <class 'geopandas.geodataframe.GeoDataFrame'>
Geometry column in stormpoints_gdf: geometry
Geometry column in dem_gdf: geometry
CRS of stormpoints_gdf: EPSG:2264
CRS of dem_gdf: EPSG:2264
Previewing the dem_gdf 'grid_code' column...
                     geometry  grid_code
0  POINT (1430025.001 654975)        783
1  POINT (1430075.001 654975)        781
2  POINT (1430125.001 654975)        770
3  POINT (1430175.001 654975)        771
4  POINT (1430225.001 654975)        771


In [40]:
# print("Starting the process...")

# # Load the GeoDataFrames
# print("Loading storm points...")
# stormpoints_gdf = gpd.read_file(stormPath)
# print(f"Loaded {len(stormpoints_gdf)} storm points.")

# print("Loading DEM points...")
# dem_gdf = gpd.read_file(demPointsPath)
# print(f"Loaded {len(dem_gdf)} DEM points.")

# # # Ensure both GeoDataFrames have the same CRS
# # print("Aligning CRS...")
# # if stormpoints_gdf.crs != target_crs:
# #     stormpoints_gdf = stormpoints_gdf.to_crs(target_crs)
# #     print(f"Reprojected storm points to {target_crs}.")
# # if dem_gdf.crs != target_crs:
# #     dem_gdf = dem_gdf.to_crs(target_crs)
# #     print(f"Reprojected DEM points to {target_crs}.")

# # Ensure valid geometries
# # print("Validating geometries...")
# # stormpoints_gdf = stormpoints_gdf[stormpoints_gdf.geometry.notnull()]
# # dem_gdf = dem_gdf[dem_gdf.geometry.notnull()]
# # stormpoints_gdf["geometry"] = stormpoints_gdf.geometry.buffer(0)
# # dem_gdf["geometry"] = dem_gdf.geometry.buffer(0)
# # print(f"Validated geometries: {len(stormpoints_gdf)} storm points and {len(dem_gdf)} DEM points remain.")

# # Perform a spatial join to find the nearest DEM point for each storm point
# print("Finding nearest DEM points for each storm point...")
# stormpoints_with_elevation = gpd.sjoin_nearest(
#     stormpoints_gdf,
#     dem_gdf[['geometry', 'grid_code']],  # Only keep geometry and grid_code columns
#     how="left",
#     distance_col="distance"
# )
# print("Nearest DEM points found and joined with storm points.")

# # Rename the `grid_code` column to `elevation`
# print("Renaming 'grid_code' to 'elevation'...")
# stormpoints_with_elevation = stormpoints_with_elevation.rename(columns={"grid_code": "elevation"})

In [50]:
# Check input GeoDataFrame types
print("Checking input GeoDataFrame types...")
print(f"stormpoints_gdf type: {type(stormpoints_gdf)}")
print(f"dem_gdf type: {type(dem_gdf)}")

# Check geometry column in inputs
print("Checking geometry columns in inputs...")
print(f"stormpoints_gdf geometry column: {stormpoints_gdf.geometry.name}")
print(f"dem_gdf geometry column: {dem_gdf.geometry.name}")

# Perform the spatial join
print("Performing spatial join...")
stormpoints_with_elevation = gpd.sjoin_nearest(
    stormpoints_gdf,
    dem_gdf[['geometry', 'grid_code']],  # Only keep geometry and grid_code columns
    how="left",
    distance_col="distance"
)

# Check output type
print("Checking output type...")
print(f"stormpoints_with_elevation type: {type(stormpoints_with_elevation)}")

# Ensure geometry column exists in output
if 'geometry' not in stormpoints_with_elevation.columns:
    raise ValueError("The output of sjoin_nearest has lost its geometry column.")

# Check if the result is still a GeoDataFrame
if not isinstance(stormpoints_with_elevation, gpd.GeoDataFrame):
    stormpoints_with_elevation = gpd.GeoDataFrame(
        stormpoints_with_elevation,
        geometry=stormpoints_with_elevation['geometry'],
        crs=stormpoints_gdf.crs
    )

# Rename the 'grid_code' column to 'elevation'
stormpoints_with_elevation = stormpoints_with_elevation.rename(columns={"grid_code": "elevation"})


Checking input GeoDataFrame types...
stormpoints_gdf type: <class 'geopandas.geodataframe.GeoDataFrame'>
dem_gdf type: <class 'geopandas.geodataframe.GeoDataFrame'>
Checking geometry columns in inputs...
stormpoints_gdf geometry column: geometry
dem_gdf geometry column: geometry
Performing spatial join...
Checking output type...
stormpoints_with_elevation type: <class 'geopandas.geodataframe.GeoDataFrame'>


In [63]:
# Verify the rename was successful
print("Columns after renaming:")
print(stormpoints_with_elevation.columns)

# Convert distance from feet to miles
stormpoints_with_elevation["distance_miles"] = stormpoints_with_elevation["distance"] / 5280

print("Previewing stormpoints_with_elevation...")
print(stormpoints_with_elevation[['geometry', 'elevation','distance_miles']].head(5))


Columns after renaming:
Index(['OBJECTID', 'Latitude', 'Longitude', 'Division', 'MaintSctn',
       'RouteClass', 'RouteQuali', 'RouteNumbe', 'RouteInven', 'Milepost',
       'GrateCount', 'LidLength', 'LidWidth', 'InvertCoun', 'lat', 'lon',
       'geometry', 'nearest_road', 'nearest_stream', 'index_right',
       'elevation', 'distance', 'distance_miles'],
      dtype='object')
Previewing stormpoints_with_elevation...
                 geometry  elevation  distance_miles
0  POINT (-78.966 34.988)        600      277.343504
1  POINT (-78.966 34.989)        600      277.343504
2  POINT (-78.966 34.989)        600      277.343504
3  POINT (-78.966 34.989)        600      277.343504
4  POINT (-78.966 34.989)        600      277.343504


In [59]:
print("CRS details:")
print(stormpoints_gdf.crs)

CRS details:
EPSG:2264


# Step Three: Export


### Results to CSV

In [43]:
outputPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\03_outputTesting\roads_streams_DEM_Test.csv"

In [46]:
output_df = stormpoints_with_elevation[['OBJECTID', 'Latitude', 'Longitude', 'nearest_road','nearest_stream','elevation']]
output_df.to_csv(outputPath)